In [ ]:
import os
import torch
import timm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import ImageFile, Image
import matplotlib.pyplot as plt

ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        try:
            path, target = self.imgs[index]
            img = Image.open(path)
            if img.mode != 'RGB':
                img = img.convert('RGB')
            if self.transform is not None:
                img = self.transform(img)
            return img, target
        except (OSError, IOError):
            print(f"[Warning] Skipping corrupted image at index {index}")
            return self.__getitem__((index + 1) % len(self.imgs))

train_dir = "/kaggle/input/ai-generated-images-vs-real-images/train"
test_dir = "/kaggle/input/ai-generated-images-vs-real-images/test"

img_size = 299  # InceptionV4 default input
batch_size = 32

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # scale to [-1, 1]
])


if __name__ == '__main__':
    train_data = SafeImageFolder(train_dir, transform=transform)
    test_data = SafeImageFolder(test_dir, transform=transform)
    print(f"Found {len(train_data)} training images and {len(test_data)} test images.")
    if len(train_data) == 0:
        raise ValueError(f"No images found in {train_dir}. Please check your data path and structure.")
    if len(test_data) == 0:
        raise ValueError(f"No images found in {test_dir}. Please check your data path and structure.")

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = timm.create_model("inception_v4", pretrained=True, num_classes=2)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    epochs = 5
    train_loss_hist, test_loss_hist = [], []
    train_acc_hist, test_acc_hist = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_corrects = torch.tensor(0, dtype=torch.float32, device=device)
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_data)
        epoch_acc = running_corrects.double() / len(train_data)
        train_loss_hist.append(epoch_loss)
        train_acc_hist.append(epoch_acc.item())

        model.eval()
        val_loss = 0.0
        val_corrects = torch.tensor(0, dtype=torch.float32, device=device)
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * images.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(test_data)
        val_acc = val_corrects.double() / len(test_data)
        test_loss_hist.append(val_loss)
        test_acc_hist.append(val_acc.item())

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} - "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    torch.save(model.state_dict(), "inceptionv4_deepfake.pth")
    print("Model weights saved as inceptionv4_deepfake.pth")

    plt.figure(figsize=(10,5))
    plt.plot(train_loss_hist, label="Train Loss")
    plt.plot(test_loss_hist, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training & Validation Loss")
    plt.show()

    plt.figure(figsize=(10,5))
    plt.plot(train_acc_hist, label="Train Acc")
    plt.plot(test_acc_hist, label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Training & Validation Accuracy")
    plt.savefig("training_curves.png")
    plt.show()
    print("training_curves.png saved")


Found 48000 training images and 12000 test images.


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (98058240 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (98806617 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (96000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr

Epoch 1/10 - Train Loss: 0.1990, Train Acc: 0.9209 - Val Loss: 0.1253, Val Acc: 0.9542
Epoch 2/10 - Train Loss: 0.1057, Train Acc: 0.9610 - Val Loss: 0.1006, Val Acc: 0.9629
Epoch 3/10 - Train Loss: 0.0692, Train Acc: 0.9740 - Val Loss: 0.1122, Val Acc: 0.9586
Epoch 4/10 - Train Loss: 0.0491, Train Acc: 0.9821 - Val Loss: 0.1206, Val Acc: 0.9563
Epoch 5/10 - Train Loss: 0.0440, Train Acc: 0.9839 - Val Loss: 0.1471, Val Acc: 0.9544
